In [1]:
import numpy as np
import os
import yaml
from lomentum.loader import read_element
from lomentum.plotting import plotter
from qdev_wrappers.cus

In [2]:
with open('sidebanding_element.yaml') as f:
    yf = yaml.safe_load(f)
    template_element = read_element(yf)

In [3]:
def print_params(instr):
    print(instr.name.upper())
    for k, v in instr.parameters.items():
        if k != 'IDN':
            print(k, ': ', v())
    for k, v in instr.submodules.items():
        if isinstance(v, InstrumentBase):
            print('\n')
            print_params(v)